In [404]:
%pip install psycopg2
%pip install python-dotenv
%pip install folium

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import psycopg2
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd

%matplotlib inline

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Connect to DB

In [424]:
load_dotenv()
conn = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    database=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    port=os.getenv("DB_PORT"),
)
cur = conn.cursor()

# Wifi Stats Data Preprocessing

In [420]:
query = "SELECT * FROM public.wifistats"
cur.execute(query)
data = cur.fetchall()

In [ ]:
wifi_df = pd.DataFrame(
    data,
    columns=[
        "label",
        "time",
        "ping_str",
        "signal (dBm)",
        "tx_bitrate_str",
        "rx_bitrate_str",
        "connected_time_str",
        "packet_loss",
    ],
)

wifi_df["connected_time (s)"] = (
    wifi_df["connected_time_str"].str.split(" ", expand=True)[0].astype(float)
)
wifi_df["tx_bitrate (MBit/s)"] = (
    wifi_df["tx_bitrate_str"].str.split(" ", expand=True)[0].astype(float)
)
wifi_df["rx_bitrate (MBit/s)"] = (
    wifi_df["rx_bitrate_str"].str.split(" ", expand=True)[0].astype(float)
)

pattern = r"min/avg/max/mdev = ([\d\.]+)/([\d\.]+)/([\d\.]+)/([\d\.]+) ms"
extracted_values = wifi_df["ping_str"].str.extract(pattern).astype(float)
extracted_values.columns = [
    "min_ping_rtt (ms)",
    "avg_ping_rtt (ms)",
    "max_ping_rtt (ms)",
    "mdev_ping_rtt (ms)",
]

wifi_df = pd.concat([wifi_df, extracted_values], axis=1)
wifi_df = wifi_df.drop(
    ["connected_time_str", "tx_bitrate_str", "rx_bitrate_str", "ping_str"], axis=1
)

wifi_df.to_csv("./data/wifistats.csv", index=False)

display(wifi_df.head())

,label,time,signal (dBm),packet_loss,connected_time (s),tx_bitrate (MBit/s),rx_bitrate (MBit/s),min_ping_rtt (ms),avg_ping_rtt (ms),max_ping_rtt (ms),mdev_ping_rtt (ms)
0,raspi-e4:5f:01:a0:50:5f,2024-10-06 01:19:52.908003,-52.0,0.0,990597.0,180.0,200.0,4.463,4.868,5.433,0.324
1,raspi-e4:5f:01:84:b2:b2,2024-10-06 01:19:52.908466,-47.0,0.0,5064318.0,65.0,72.2,3.467,4.650,5.751,0.831
2,raspi-e4:5f:01:8d:c9:0d,2024-10-06 01:19:52.908851,-47.0,0.0,112280.0,65.0,72.2,4.060,5.145,7.381,1.181
3,raspi-e4:5f:01:a7:b1:e5,2024-10-06 01:19:52.909239,-57.0,0.0,334005.0,200.0,200.0,4.035,6.185,8.064,1.328
4,raspi-e4:5f:01:ad:15:f9,2024-10-06 01:19:52.909629,-46.0,0.0,5155733.0,72.2,72.2,2.903,4.598,5.778,0.995


# Device Data Preprocessing

In [425]:
query = "SELECT * FROM public.devices"
cur.execute(query)
data = cur.fetchall()

In [426]:
devices_df = pd.DataFrame(
    data,
    columns=[
        "label",
        "ethernet_mac",
        "wlan_mac",
        "location",
        "tags",
        "comment",
        "uptime",
        "wireless_bytes",
        "wired_bytes",
        "last_seen",
    ],
)

devices_df.to_csv("./data/devices.csv", index=False)

display(devices_df.head())

,label,ethernet_mac,wlan_mac,location,tags,comment,uptime,wireless_bytes,wired_bytes,last_seen
0,raspi-dc:a6:32:d7:6e:64,dc:a6:32:d7:6e:64,dc:a6:32:d7:6e:65,None,None,None,"18:11:40 up 206 days, 17:13, 0 users, load ...",7.723500e+07,1.246234e+10,2023-11-15 18:11:55.004674
1,raspi-e4:5f:01:72:89:99,e4:5f:01:72:89:99,e4:5f:01:72:89:9a,None,None,None,"17:43:43 up 154 days, 23:15, 0 users, load ...",5.499737e+07,7.350356e+09,2023-09-27 17:43:58.456155
2,raspi-e4:5f:01:72:a3:e5,e4:5f:01:72:a3:e5,e4:5f:01:72:a3:e6,None,None,None,None,NaN,NaN,NaT
3,raspi-e4:5f:01:9c:25:7d,e4:5f:01:9c:25:7d,e4:5f:01:9c:25:7e,"Jamala, 2210","[UCSB, Manzanita, Jamala]",None,"21:18:48 up 579 days, 1:31, 0 users, load ...",1.438647e+10,0.000000e+00,2024-11-23 21:19:03.508499
4,raspi-e4:5f:01:56:d6:ce,e4:5f:01:56:d6:ce,e4:5f:01:56:d6:cf,None,None,None,"18:11:40 up 206 days, 17:06, 2 users, load ...",1.672108e+09,1.082823e+10,2023-11-15 18:11:55.017829


# Extra Info Data Preprocessing

In [427]:
query = "SELECT * FROM public.extra_info"
cur.execute(query)
data = cur.fetchall()

In [429]:
extra_info_df = pd.DataFrame(
    data,
    columns=[
        "label",
        "comment",
        "contact_person",
        "latitude",
        "longitude",
    ],
)

extra_info_df["latitude"].replace("None", np.nan, inplace=True)
extra_info_df["longitude"].replace("None", np.nan, inplace=True)
extra_info_df = extra_info_df.dropna(subset=["latitude", "longitude"]).reset_index(
    drop=True
)
extra_info_df = extra_info_df.drop(["comment", "contact_person"], axis=1)

extra_info_df.to_csv("./data/extra_info.csv", index=False)

display(extra_info_df.head())

,label,latitude,longitude
0,raspi-e4:5f:01:75:6b:2c,34.41773457866324,-119.8546902810392
1,raspi-e4:5f:01:8e:27:aa,34.409574095820076,-119.85180308648167
2,raspi-e4:5f:01:a7:b2:7e,34.409548306773345,-119.85153880818854
3,raspi-e4:5f:01:75:54:04,34.418302607166105,-119.85669070847874
4,raspi-e4:5f:01:9b:84:c4,34.408679667265304,-119.85184124971644


# Close DB connection

In [416]:
cur.close()
conn.close()